In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import datapane as dp
from sklearn import linear_model

# SCRAPING ALL WEEK-8 DATA GOING BACK TO 1978

In [7]:
year = 1978

total_week_8_teams, total_week_8_ptDiff, total_week_8_wins = [], [], []
total_week_16_teams, total_week_16_ptDiff, total_week_16_wins = [], [], []

while year < 2021:
    week = 8
    while week < 18:
        
        if week == 8:
            
            url = ("https://www.pro-football-reference.com/boxscores/standings.cgi?week=" + str(week) + 
                    "&year=" + str(year) + "&wk_league=NFL")

            r = requests.get(url,timeout=2.5)
            r_html = r.text

            soup = BeautifulSoup(r_html, 'html.parser')
            
            week_8_teams = soup.find_all('th', attrs={"class": "left"})
            week_8_teams = [str(l) for l in week_8_teams]
            
            for l in week_8_teams:
                left, right = '.htm">', '</a>'
                if left in l and right in l:
                    total_week_8_teams.append(l[l.index(left)+len(left):l.index(right)])
                else:
                    continue
                    
            week_8_table = soup.find_all("td")
            week_8_table = [str(l) for l in week_8_table]
            
            for l in week_8_table:
                if 'wins">' in l and '</td>' in l:
                    total_week_8_wins.append(l[l.index('wins">')+len('wins">'):l.index('</td>')])
                if 'points_diff">' in l and '</td>' in l:
                    total_week_8_ptDiff.append(l[l.index('points_diff">')+len('points_diff">'):l.index('</td>')])
                else:
                    continue
            week += 9
        
        if week == 17:
            
            url = ("https://www.pro-football-reference.com/boxscores/standings.cgi?week=" + str(week) + 
                    "&year=" + str(year) + "&wk_league=NFL")

            r = requests.get(url,timeout=2.5)
            r_html = r.text

            soup = BeautifulSoup(r_html, 'html.parser')
            
            week_16_teams = soup.find_all('th', attrs={"class": "left"})
            week_16_teams = [str(l) for l in week_16_teams]
            
            for l in week_16_teams:
                left, right = '.htm">', '</a>'
                if left in l and right in l:
                    total_week_16_teams.append(l[l.index(left)+len(left):l.index(right)])
                else:
                    continue
                    
            week_16_table = soup.find_all("td")
            week_16_table = [str(l) for l in week_16_table]
            
            for l in week_16_table:
                if 'wins">' in l and '</td>' in l:
                    total_week_16_wins.append(l[l.index('wins">')+len('wins">'):l.index('</td>')])
                if 'points_diff">' in l and '</td>' in l:
                    total_week_16_ptDiff.append(l[l.index('points_diff">')+len('points_diff">'):l.index('</td>')])
                else:
                    continue 
            week += 1
    if year == 1981:
        year += 2
    else:
        year += 1

## Lining up lists so teams are in proper order

In [8]:
new_total_wins, new_total_teams = [], []

for i in total_week_8_teams:
    if i in total_week_16_teams:
        new_total_teams.append(i)
        new_total_wins.append(total_week_16_wins[total_week_16_teams.index(i)])
        total_week_16_teams[total_week_16_teams.index(i)] = ''

## DataFrame

In [114]:
df = pd.DataFrame(data={"Team": new_total_teams, "Wk 8 Pts Margin": total_week_8_ptDiff, "Wk 8 Wins": total_week_8_wins,'Wk 17 Wins': total_week_16_wins})

In [115]:
total_week_16_wins = [float(l) for l in total_week_16_wins]
total_week_8_ptDiff = [float(l) for l in total_week_8_ptDiff]

In [116]:
X = df[['Wk 8 Pts Margin']]
y = df['Wk 8 Wins']

scorig_margin = df['Wk 8 Pts Margin'].tolist()
week8actual_wins = df['Wk 8 Wins'].tolist()
week8actual_wins = [float(l) for l in week8actual_wins]

week8predicted_wins = []

for i in scorig_margin:
    
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    week8predicted_wins.append(round(float(regr.predict([[i]])), 2))


#df['Week 8 Predicted Wins'] = week8predicted_wins
week8diff_list = []

for i, k in zip(week8actual_wins, week8predicted_wins):
    diff = str(round(i-k, 2))
    if float(diff) > 0:
        diff = '+' + str(diff)

    week8diff_list.append(diff)

week8diff_list = [float(l) for l in week8diff_list]

df.insert(3, 'Wk 8 Expected', week8predicted_wins)
#df.insert(4, 'Week 8 Difference', week8diff_list)


In [117]:
X = df[['Wk 8 Pts Margin']]
y = df['Wk 17 Wins']

scorig_margin = df['Wk 8 Pts Margin'].tolist()
actual_wins = df['Wk 17 Wins'].tolist()

predicted_wins = []

for i in scorig_margin:
    
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    predicted_wins.append(round(float(regr.predict([[i]])), 2))
                                                                               

In [118]:
week_17_predicted = []
for i, k in zip(predicted_wins, week8diff_list):
    week_17_predicted.append(i + k)
df['Wk 17 Predicted'] = week_17_predicted

In [119]:
df['Wk 17 Predicted'] = week_17_predicted

In [120]:
diff_list = []
for i, k in zip(week_17_predicted, actual_wins):
    diff_list.append(k - i)
df['Difference'] = diff_list

In [121]:
df

,Team,Wk 8 Pts Margin,Wk 8 Wins,Wk 8 Expected,Wk 17 Wins,Wk 17 Predicted,Difference
0,New England Patriots,39.0,6,4.79,11.0,10.95,0.05
1,Miami Dolphins,55.0,5,5.23,11.0,10.24,0.76
2,New York Jets,43.0,5,4.90,8.0,10.02,-2.02
3,Buffalo Bills,-30.0,3,2.88,5.0,6.71,-1.71
4,Baltimore Colts,-92.0,3,1.17,5.0,5.59,-0.59
...,...,...,...,...,...,...,...
1264,Atlanta Falcons,-15.0,2,3.30,4.0,5.97,-1.97
1265,Seattle Seahawks,41.0,6,4.84,12.0,10.99,1.01
1266,Arizona Cardinals,57.0,5,5.29,10.0,10.27,-0.27
1267,Los Angeles Rams,41.0,5,4.84,8.0,9.99,-1.99


In [25]:
diff_list_float = [float(l) for l in diff_list]

abs_vals = [abs(x) for x in diff_list_float]

avg_difference = sum(abs_vals) / len(diff_list_float)

avg_difference

1.1645153664302594

## Model

In [26]:
model = np.polyfit(df['Week 8 Pts Margin'], df['Actual Wins'], 1)

In [27]:
predict = np.poly1d(model)

r2 = r2_score(total_week_16_wins, predict(total_week_8_ptDiff))

In [28]:
np.sqrt(r2)

0.8016873170281702

In [93]:
model

array([0.04570157, 7.95981087])

## Current week 8 standings

In [4]:
week, year = 8, 2021
current_wins, current_ptDiff, current_teams = [], [], []

url = ("https://www.pro-football-reference.com/boxscores/standings.cgi?week=" + str(week) + 
        "&year=" + str(year) + "&wk_league=NFL")

r = requests.get(url,timeout=2.5)
r_html = r.text

soup = BeautifulSoup(r_html, 'html.parser')

current_week_8_teams = soup.find_all('th', attrs={"class": "left"})
current_week_8_teams = [str(l) for l in current_week_8_teams]

current_week_8_table = soup.find_all("td")
current_week_8_table = [str(l) for l in current_week_8_table]

for l in current_week_8_teams:
    left, right = '.htm">', '</a>'
    if left in l and right in l:
        current_teams.append(l[l.index(left)+len(left):l.index(right)])
    else:
        continue

for l in current_week_8_table:
    if 'wins">' in l and '</td>' in l:
        current_wins.append(l[l.index('wins">')+len('wins">'):l.index('</td>')])
    if 'points_diff">' in l and '</td>' in l:
        current_ptDiff.append(l[l.index('points_diff">')+len('points_diff">'):l.index('</td>')])
    else:
        continue

## Projecting Current Season Win Totals

In [159]:
new_df = pd.DataFrame(data={"Team": current_teams, "Wk 8 Pts Margin": current_ptDiff})
new_df

,Team,Wk 8 Pts Margin
0,Buffalo Bills,120.0
1,New England Patriots,42.0
2,New York Jets,-92.0
3,Miami Dolphins,-95.0
4,Baltimore Ravens,23.0
5,Cincinnati Bengals,58.0
6,Pittsburgh Steelers,-10.0
7,Cleveland Browns,3.0
8,Tennessee Titans,32.0
9,Indianapolis Colts,17.0


In [85]:
current_ptDiff = [float(l) for l in current_ptDiff]

In [86]:
projected_wins = []

for i in current_ptDiff:
    
    X = df[['Week 8 Pts Margin']]
    y = df['Actual Wins']

    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    
    prediction = regr.predict([[i]])
    
    projected_wins.append(round(float(prediction), 2))

In [87]:
wins_thru_17 = [round((i/16)*17, 2) for i in projected_wins]

In [88]:
X = new_df[['Wk 8 Pts Margin']]
y = current_wins

current_predicted_wins = []

for i in current_ptDiff:
    
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    current_predicted_wins.append(round(float(regr.predict([[i]])), 2))
    
week8diff_list = []
current_wins = [float(l) for l in current_wins]
current_predicted_wins = [float(l) for l in current_predicted_wins]

for i, k in zip(current_wins, current_predicted_wins):
    diff = str(round(i-k, 2))
    if float(diff) > 0:
        diff = '+' + str(diff)

    week8diff_list.append(diff)

week8diff_list = [float(l) for l in week8diff_list]

new_df.insert(2, 'Wk 8 Wins', current_wins)
#new_df.insert(3, 'Wk 8 Expected Wins', current_predicted_wins)

In [89]:
week_17_predicted = []
for i, k in zip(wins_thru_17, week8diff_list):
    week_17_predicted.append(i + k)
new_df['Wk 17 Predicted Wins'] = week_17_predicted

In [90]:
new_df

,Team,Wk 8 Pts Margin,Wk 8 Wins,Wk 17 Predicted Wins
0,Buffalo Bills,120.0,5.0,12.37
1,New England Patriots,42.0,4.0,9.60
2,New York Jets,-92.0,2.0,4.56
3,Miami Dolphins,-95.0,1.0,3.49
4,Baltimore Ravens,23.0,5.0,10.16
5,Cincinnati Bengals,58.0,5.0,10.96
6,Pittsburgh Steelers,-10.0,4.0,8.42
7,Cleveland Browns,3.0,4.0,8.72
8,Tennessee Titans,32.0,6.0,11.37
9,Indianapolis Colts,17.0,3.0,8.04


In [91]:
table = dp.DataTable(new_df)
dp.Report(table).upload(name='Projected Win Totals for the 2021 NFL Season')

Uploading report and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a single report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Report successfully uploaded, click [here](https://datapane.com/u/wmblack23/reports/Y3YVgz7/projected-win-totals-for-the-2021-nfl-season/) to view and share your report

In [141]:
fig = px.scatter(df, x = df['Wk 8 Pts Margin'], y = df['Wk 17 Wins'], color = df['Wk 8 Pts Margin'],
          size = df['Wk 17 Wins'], size_max=17, trendline='ols', template='gridon',
          title='NFL Win Totals & Week 8 Scoring Margins | 1978-2020',
          hover_data = [df['Team']])

fig

In [149]:
graph = dp.Plot(fig)
dp.Report(graph).upload(name='NFL Win Totals & Scoring Margins | 1978-2020')

Uploading report and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a single report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Report successfully uploaded, click [here](https://datapane.com/u/wmblack23/reports/W3Dzjzk/nfl-win-totals-scoring-margins-1978-2020/) to view and share your report

In [148]:
df.drop(columns = 'Wk 8 Expected', inplace=True)

In [150]:
table = dp.DataTable(df)
dp.Report(table).upload(name='Week 8 Scoring Margin compared to End-of-Season Wins')

Uploading report and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a single report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Report successfully uploaded, click [here](https://datapane.com/u/wmblack23/reports/d7dXnG7/week-8-scoring-margin-compared-to-end-of-season-wins/) to view and share your report